# I RNN (LSTM)

[torchtext](https://pytorch.org/text/stable/index.html)

In [ ]:
import sys
import time
import torch
import random
import torchtext
import pandas as pd
import torch.nn.functional as F

torch.backends.cudnn.deterministic = True

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [ ]:
RANDOM_SEED     = 123
VOCABULARY_SIZE = 20000
LEARNING_RATE   = 0.005
BATCH_SIZE      = 128
NUM_EPOCHS      = 15
EMBEDDING_DIM   = 128
HIDDEN_DIM      = 256
NUM_CLASSES     = 2
DEVICE          = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
torch.manual_seed(RANDOM_SEED)

## Duomenų rinkinys

Apdorotas IMDB filmų peržiūros duomenų rinkinys kurio orginalas yra iš (https://ai.stanford.edu/~amaas/data/sentiment/).csv formato faile

In [ ]:
shutil.copy('/content/drive/My Drive/Colab Notebooks/datasets/movie_data.csv.gz', 'movie_data.csv.gz')

In [ ]:
!gunzip -f movie_data.csv.gz 

In [ ]:
df = pd.read_csv('movie_data.csv')
df.head()

In [ ]:
df.columns = ['TEXT_COLUMN_NAME', 'LABEL_COLUMN_NAME']
df.to_csv('movie_data.csv', index=None)

In [ ]:
df = pd.read_csv('movie_data.csv')
df.head()

In [ ]:
del df

## Parengiame duomenų rinkinį su `torchtext`

Tam naudosime python biblioteka kuri vadinasi <code>**[spacy](https://spacy.io/usage)**</code>. Tai labai populiari NLP (natural language processing) biblioteka ir mes naudosime iš šios bibliotekos `tokenizer`

In [ ]:
!pip install setuptools wheel spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
TEXT = torchtext.legacy.data.Field(
    tokenize='spacy', # numatytasis tarpų padalijimas
    tokenizer_language='en_core_web_sm' # anglų kalbos žodynas
)

# Etikečių apdorojimo apibrėžimas
LABEL = torchtext.legacy.data.LabelField(dtype=torch.long)

In [ ]:
fields = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]

dataset = torchtext.legacy.data.TabularDataset(
    path='movie_data.csv', 
    format='csv',
    skip_header=True, 
    fields=fields
)

In [ ]:
# train_data, valid_data, test_data = dataset.split(
#     split_ratio=[0.7, 0.1, 0.2],
#     random_state=random.seed(RANDOM_SEED))

# print(f'treniravimo duomenys: {len(train_data)}')
# print(f'validacijos duomenys: {len(valid_data)}')
# print(f'testavimo duomenys  : {len(test_data)}')

In [ ]:
train_data, test_data = dataset.split(
    split_ratio=[0.8, 0.2],
    random_state=random.seed(RANDOM_SEED))

print(f'treniravimo duomenys: {len(train_data)}')
print(f'testavimo duomenys  : {len(test_data)}')

In [ ]:
train_data, valid_data = train_data.split(
    split_ratio=[0.85, 0.15],
    random_state=random.seed(RANDOM_SEED))

print(f'treniravimo duomenys: {len(train_data)}')
print(f'testavimo duomenys  : {len(valid_data)}')

python [vars](https://docs.python.org/3/library/functions.html#vars)

In [ ]:
print(vars(train_data.examples[0]))

## Sukuriame žodyną

kurkite žodyną pagal dažniausius žodžius naudojant `VOCABULARY_SIZE = 20000`

In [ ]:
TEXT.build_vocab(train_data, max_size=VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'Žodyno dydis   : {len(TEXT.vocab)}')
print(f'Klasių skaičius: {len(LABEL.vocab)}')

 - 20002 dėl `<unk>` ir `<pad>` tokenų.
 - PyTorch RNN gali susidoroti su skirtingais ilgiais dėl dinaminių grafikų, tačiau užpildymas `padding` yra būtinas norint užpildyti tokio pat ilgio sekas tam tikroje mini pakuotėje, kad galėtume jas saugoti masyve.

<code>**Dažniausi žodžiai:**</code>

In [ ]:
print(TEXT.vocab.freqs.most_common(20))

<code>**Tokenai atitinkantys pirmuosius 10 indeksų (0,1,...,9):**</code>

In [ ]:
print(TEXT.vocab.itos[:10])

<code>**Konvertuojame tekstą į skaičių:**</code>

In [ ]:
print(TEXT.vocab.stoi['the'])

<code>**Etiketės**</code>

In [ ]:
print(LABEL.vocab.stoi)

<code>**Etikečių dažnis**</code>

In [ ]:
LABEL.vocab.freqs

## Duomenų krautuvas

In [ ]:
train_loader, valid_loader, test_loader = \
    torchtext.legacy.data.BucketIterator.splits(
        (train_data, valid_data, test_data),
         batch_size=BATCH_SIZE,
         sort_within_batch=False,
         sort_key=lambda x: len(x.TEXT_COLUMN_NAME),
         device=DEVICE
    )

In [ ]:
print('Treniravimas')
for batch in train_loader:
    print(f'Teksto matricos dydis: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Etikečių vektoriaus dydis: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nValidacija:')
for batch in valid_loader:
    print(f'Teksto matricos dydis: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Etikečių vektoriaus dydis: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nTestavimas:')
for batch in test_loader:
    print(f'Teksto matricos dydis: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Etikečių vektoriaus dydis: {batch.LABEL_COLUMN_NAME.size()}')
    break


In [ ]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        #self.rnn = torch.nn.RNN(embedding_dim,
        #                        hidden_dim,
        #                        nonlinearity='relu')
        self.rnn = torch.nn.LSTM(embedding_dim,
                                 hidden_dim)        
        
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        

    def forward(self, text):
        # text dim: [sentence length, batch size]
        
        embedded = self.embedding(text)
        # embedded dim: [sentence length, batch size, embedding dim]
        
        output, (hidden, cell) = self.rnn(embedded)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]

        hidden.squeeze_(0)
        # hidden dim: [batch size, hidden dim]
        
        output = self.fc(hidden)
        return output


In [ ]:
class RNN_packed(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        self.rnn       = torch.nn.LSTM(embedding_dim, hidden_dim)        
        self.fc        = torch.nn.Linear(hidden_dim, output_dim)
        

    def forward(self, text, text_length):
        # teksto matmenys: [sakinio ilgis, pakuotės dydis]
        
        embedded = self.embedding(text)
        # ebedded dim: [sakinio ilgis, pakuotės dydis, iterptin dim]
        
        ## nauja
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_length.to('cpu'))
        
        packed_output, (hidden, cell) = self.rnn(packed)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]

        hidden.squeeze_(0)
        # hidden dim: [batch size, hidden dim]
        
        output = self.fc(hidden)
        return output

In [ ]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim=len(TEXT.vocab),
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=NUM_CLASSES # gali naudoti 1 dvejetainei klasifikacijai
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)


In [ ]:
torch.manual_seed(RANDOM_SEED)
model = RNN_packed(input_dim=len(TEXT.vocab),
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=NUM_CLASSES # could use 1 for binary classification
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [ ]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100


     

In [ ]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)

        ### FORWARD AND BACK PROP
        logits = model(text)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')


In [ ]:
import spacy


nlp = spacy.blank("en")

def predict_sentiment(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    return prediction[0][0].item()

print('Probability positive:')
predict_sentiment(model, "This is such an awesome movie, I really love it!")

In [ ]:
print('Probability negative:')
1-predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")